In [2]:
import preprocessor as p
import json
import pandas as pd
import csv
import re #regular expression
import string


## Reformating the data

In the next sections we are ging to transform the tweets from json to Pandas dataframe. We need to determin the column names.

Since we just want to run an NLP task we are going to just use the text of each tweet

In [3]:
with open('db/tweets.json') as f:
  tweets = json.load(f)

In [7]:
pos_list = []
neg_list = []

for i in range(1, len(tweets)):
    if tweets[i]["label"] == 1 :
        pos_list.append(p.clean(tweets[i]["text"]))
    elif tweets[i]["label"] == 0:
        neg_list.append(p.clean(tweets[i]["text"]))


## NLP tasks


In [51]:
import nltk
#nltk.download('stopwords')
# nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Ashkan R.
[nltk_data]     Nejad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [8]:
from nltk.corpus import stopwords 
from nltk.tokenize import sent_tokenize, word_tokenize , RegexpTokenizer

# incase of seeing unavailable package error uncomment section above

stop_words = set(stopwords.words('english')) 

tmp = []
for i in range(1, len(pos_list)):
    tmp.extend(word_tokenize(pos_list[i]))
    
for i in range(1, len(neg_list)):
    tmp.extend(word_tokenize(neg_list[i]))
    


filtering all the uncessary and duplicated symbols like . , ! (using ASCII code)

In [9]:
corpus = []
for i in range(1, len(tmp)):
    if 64 <ord(tmp[i][0])< 123:
            corpus.append(tmp[i])

corpus = set(corpus)
corpus = list(corpus)

len(corpus)
tokenized_text= [sentence.split() for sentence in corpus]

corpus


['Purple',
 'EVEN',
 'Upcoming',
 'of',
 'seeing',
 'about',
 'hear',
 'Blue',
 'deal',
 'VIDEO',
 'been',
 'new',
 'no',
 'stabilizati…',
 'fast',
 'show',
 'don',
 'make',
 'more',
 'k',
 'stepping',
 'Workers',
 'let',
 'highest-end',
 'NOT',
 'Know',
 'Go',
 'this',
 'listening',
 'Says',
 'screen',
 'Adapters',
 'model',
 'if',
 'need',
 'phone',
 'Earphone',
 'trick',
 'I',
 'vs',
 'means',
 'available',
 'OLED',
 'box',
 'the',
 'upgrade',
 'off',
 'little',
 'NEW',
 'We',
 'from',
 'computer',
 'Want',
 'shit',
 'Retweets',
 'take',
 'could',
 'Feature',
 'a…',
 'Camera',
 'classy',
 'Self-isolate',
 'pow…',
 'junk',
 'Strap',
 's',
 'can',
 'connect',
 'just',
 'years',
 'shift',
 'Meet',
 'Your',
 'Sensor-Shift',
 'cheat',
 'do',
 'AirPods',
 'Apple',
 'Ring',
 'Pros',
 'Under',
 'Protective',
 'end',
 'Wit…',
 'Production…',
 'series',
 'At',
 'still',
 'actually',
 'regular',
 'm',
 'Lavanda',
 'oled',
 'resurrect',
 'Make',
 'eected',
 'Now',
 'Secured',
 'Say',
 'release'

## Building Bag-of-words

The data is going to be saved in a dataframe in the spark-friendly format

In [17]:
import numpy as numpy

# feats = []
df = pd.DataFrame(columns=['features', 'label'])

for sentence in pos_list:
    words = word_tokenize(sentence)  
    bag_vector = numpy.zeros(len(corpus))
    for word in words:
        for i,w in enumerate(corpus):
            if word == w:
#                 print(word, w, i, corpus[i])                
                bag_vector[i] += 1 

#     feats.append(bag_vector)
    df = df.append({'features': bag_vector, 'label': 1}, ignore_index=True)
    
for sentence in neg_list:
    words = word_tokenize(sentence)  
    bag_vector = numpy.zeros(len(corpus))
    for word in words:
        for i,w in enumerate(corpus):
            if word == w:
                bag_vector[i] += 1 
#     feats.append(bag_vector)
    df = df.append({'features': bag_vector, 'label': 0}, ignore_index=True)

Saving data into csv for later uses

In [167]:
import csv
with open("feats.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(feats)

Now we have all our tweet texts vectorized and ready for machine learning process

## Used links:

[1] https://www.freecodecamp.org/news/an-introduction-to-bag-of-words-and-how-to-code-it-in-python-for-nlp-282e87a9da04/
